In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as sts
import numpy as np

parks = pd.read_csv("../Resources/state-park-facility-points-1.csv")
attendance = pd.read_csv("../Resources/State_Park_Annual_Attendance_Figures_by_Facility___Beginning_2003.csv")
income = pd.read_csv("../Resources/Median Household Income by County.csv")
population = pd.read_csv("../Resources/Population Estimates by County.csv")

In [2]:
parks = parks.drop(columns=['Facility URL', 'Golf URL', 'Nature Center URL', 'Location']).fillna('N')

In [3]:
parks = parks.sort_values(by='Name')
parks
parks.to_csv(r'processed/StatePark.csv', index = False, header=True)

In [4]:
att2019 = attendance[attendance["Year"] == 2019]
att2019.rename(columns={'Facility': 'Name'}, inplace=True)
att2019 = att2019.sort_values(by = 'Name')
grouped_att = att2019.groupby(['County'])['Attendance'].sum()
grouped_county = att2019.groupby(['County'])['County']

park_att = pd.DataFrame({'County': grouped_county , 'Attendance': grouped_att})

# park_att['County']= park_att.index
# park_att.reset_index(drop=True, inplace=True)
# df['text_new'] = df['text'].str.split('::').str[0]

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\dtypes\cast.py:1302: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  v = np.array(v, copy=False)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\dtypes\cast.py:1672: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'd

In [5]:
income['Median Household Income by County'] = income['Median Household Income by County'].str.replace('County', '')
income.drop(income.tail(1).index,inplace=True)
income.drop(income.head(1).index,inplace=True)
income.rename(columns={'Median Household Income by County':'County', 'Unnamed: 1': 'Income', 'Unnamed: 2': 'Margin of Error'}, inplace=True)
income.to_csv (r'processed/income.csv', index = False, header=True)

In [6]:
population['Population Estimates by County'] = population['Population Estimates by County'].str.replace('County', '')
population.drop(income.tail(1).index,inplace=True)
population.drop(income.head(1).index,inplace=True)
population.rename(columns={'Population Estimates by County':'County', 'Unnamed: 1': 'Population'}, inplace=True)

In [7]:
population.to_csv (r'processed/population.csv', index = False, header=True)

In [8]:
att2019.to_csv (r'processed/2019_Attendance.csv', index = False, header=True)
park_att.to_csv (r'processed/park_county_attendance.csv', index = False, header=True)


In [9]:
import sqlite3 as sql
cleaned_data = pd.read_csv("processed/county_attendance_income_pop.csv")
conn = sql.connect('newyork.db')

In [10]:
conn.execute("DROP TABLE IF EXISTS county_data")

conn.execute('''CREATE TABLE county_data (
    County TEXT PRIMARY KEY,  
    Attendance INT, 
    Income INT, 
    Population INT
)''')

In [11]:
# cleaned_data['Income'] = cleaned_data['Income'].str.replace('$', '')
# cleaned_data['Income'] = cleaned_data['Income'].str.replace(',', '')
# cleaned_data['Population'] = cleaned_data['Population'].str.replace(',', '')
# cleaned_data = cleaned_data.drop(columns=['Margin of Error'])

cleaned_data
cleaned_data.to_sql('county_data', conn, if_exists='append', index= False)
cleaned_data.to_csv('county_data', index= False)
explore = pd.read_sql("SELECT * FROM county_data", conn)
explore

,County,Attendance,Income,Population
0,Albany,692080,66252,305506
1,Allegany,50133,48412,46091
2,Bronx,339868,40088,1418207
3,Broome,252508,52226,190488
4,Cattaraugus,1521210,48703,76117
...,...,...,...,...
57,Washington,8588,57258,61204
58,Wayne,80599,59449,89918
59,Westchester,2351019,96610,967506
60,Wyoming,27031,58052,39859


In [12]:
yearly_attendance = pd.read_csv("../Resources/State_Park_Annual_Attendance_Figures_by_Facility___Beginning_2003.csv")
yearly_attendance = yearly_attendance.sort_values(by='Year').drop(columns=['OPRHP Region', 'County'])

yearly_attendance.to_csv (r'processed/yearly_attendance.csv', index = False, header=True)
